### Statistical Analysis & Understanding Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import (encode_features, scale_features,
                   look_for_outliers,handle_outliers_iqr,
                   treat_skewness)
# from google.colab import drive

In [ ]:
# drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("data/students_train.csv")
df.head()

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

In [ ]:
#Summarize categorical columns:
for col in df.select_dtypes(include='object').columns:
    print(f"Value counts for {col}:\n{df[col].value_counts()}\n")

In [ ]:
for col in df.columns:
        num_unique = df[col].nunique()
        print(f"Column '{col}' has {num_unique} unique values.")

In [ ]:
print(f"Number of duplicate rows: {df.duplicated().sum()}")
# View the duplicated rows
duplicated_rows = df[df.duplicated()]
print("\nDuplicated rows:")
print(duplicated_rows)

In [ ]:
# 1. Descriptive statistics for numeric columns
numeric_desc = df.describe()
print("Descriptive Statistics (Numeric Columns):")
print(numeric_desc)

In [ ]:
# 2. Frequency distribution for top 10 values in categorical variables
categorical_cols = df.select_dtypes(include='object').columns
print("\nFrequency Distribution (Top 10 per Categorical Column):")
for col in categorical_cols:
    print(f"\nTop values in '{col}':")
    print(df[col].value_counts().head(10))

In [ ]:
# 3. Correlation matrix for numerical features
print("\nCorrelation Matrix:")
correlation_matrix = df.corr(numeric_only=True)
print(correlation_matrix)

In [ ]:
# 4. Distribution plots (Histograms) for numerical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns

plt.figure(figsize=(15, 12))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(3, 3, i)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Histogram: {col}')
plt.tight_layout()
plt.show()

In [ ]:
# 5. Boxplots for outlier detection
plt.figure(figsize=(15, 12))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot: {col}')
plt.tight_layout()
plt.show()

In [ ]:
# 6. Bar charts for top categories
plt.figure(figsize=(18, 18))
for i, col in enumerate(categorical_cols[:9], 1):  # Limit to 9 for layout
    plt.subplot(3, 3, i)
    df[col].value_counts().head(10).plot(kind='bar')
    plt.title(f'Bar Chart: {col}')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Preprocessing

In [ ]:
# Step 1: Drop rows with any missing values
df_cleaned = df.dropna()

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
df_cleaned.head()

In [ ]:
# Step 2: Standardize text-based categorical fields
df_cleaned['Name'] = df_cleaned['Name'].str.title()
df_cleaned['Gender'] = df_cleaned['Gender'].str.capitalize()
df_cleaned['Medical Condition'] = df_cleaned['Medical Condition'].str.capitalize()
df_cleaned['Doctor'] = df_cleaned['Doctor'].str.title()
df_cleaned['Hospital'] = df_cleaned['Hospital'].str.title()
df_cleaned['Insurance Provider'] = df_cleaned['Insurance Provider'].str.title()
df_cleaned['Medication'] = df_cleaned['Medication'].str.capitalize()
df_cleaned['Test Results'] = df_cleaned['Test Results'].str.capitalize()
df_cleaned['Admission Type'] = df_cleaned['Admission Type'].str.capitalize()


In [ ]:
# Step 3: Convert date columns to datetime
df_cleaned['Date of Admission'] = pd.to_datetime(df_cleaned['Date of Admission'], errors='coerce')
df_cleaned['Discharge Date'] = pd.to_datetime(df_cleaned['Discharge Date'], errors='coerce')

In [ ]:
# Calculate Length of Stay
df_cleaned['Length of Stay'] = (df_cleaned['Discharge Date'] - df_cleaned['Date of Admission']).dt.days

# Reorder columns: insert 'Length of Stay' before 'Target'
target_index = df_cleaned.columns.get_loc('Test Results')
cols = list(df_cleaned.columns)
# Move 'Length of Stay' to the position before 'Target'
cols.insert(target_index, cols.pop(cols.index('Length of Stay')))
df_cleaned = df_cleaned[cols]

In [ ]:
df_cleaned.head()

In [ ]:
#Checking the percentage of the missing data
pd.set_option('display.max_rows', None)
missing_percentage = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100
missing_df = pd.DataFrame({'Feature': df_cleaned.columns, 'Missing Percentage': missing_percentage})
print(missing_df)

In [ ]:
df_cleaned.duplicated().sum()

In [ ]:
df_cleaned.isnull().sum()

In [ ]:
look_for_outliers(df_cleaned)

In [ ]:
# List of numerical features
numerical_features = df_cleaned.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Apply the function to handle outliers
df = handle_outliers_iqr(df_cleaned, numerical_features)

In [ ]:
look_for_outliers(df)

In [ ]:
pd.set_option('display.max_rows', None)  #this line to show all of the records
df_dtypes = pd.DataFrame({"Feature": df.columns, "Data Type": df.dtypes})
print(df_dtypes)
pd.reset_option('display.max_rows')

In [ ]:
# Drop ID and Name (optional)
df.drop(columns=['ID', 'Name', 'Room Number'], inplace=True)

X_encoded, y_encoded = encode_features(df, target_col="Test Results")
df = pd.concat([X_encoded, y_encoded.rename("Test Results")], axis=1)

In [ ]:
pd.set_option('display.max_rows', None)  #this line to show all of the records
df_dtypes = pd.DataFrame({"Feature": df.columns, "Data Type": df.dtypes})
print(df_dtypes)
pd.reset_option('display.max_rows')

In [ ]:
df.head()

In [ ]:
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_features = [col for col in numerical_features if col != 'Test Results']

treated_df, skewed_features, transformation_details = treat_skewness(df, numerical_features)

In [ ]:
df_scaled, scaled_cols = scale_features(df, target_col='Test Results', scaler_type='standard')